In [1]:
import sympy as sym
import numpy as np
sym.init_printing()

In [2]:
# Mass and moment of inertia of the bus
mass = 0.86    # The mass of the quadrotor in kg
Ixx, Iyy, Izz, l, h = sym.symbols('Ixx, Iyy, Izz, l, h')
g = 9.81

In [3]:
# Define position
px_inW, py_inW, pz_inW = sym.symbols('px, py, pz')
p_inW = sym.Matrix([[px_inW],
                    [py_inW],
                    [pz_inW]])

# Define velocity
vx_inB, vy_inB, vz_inB = sym.symbols('vx, vy, vz')
v_inB = sym.Matrix([[vx_inB],
                    [vy_inB],
                    [vz_inB]])

# Define roll (phi), pitch (theta), and yaw (psi) angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# Define angular velocities
wx_inB, wy_inB, wz_inB = sym.symbols('omega_x, omega_y, omega_z')
w_inB = sym.Matrix([[wx_inB],
                    [wy_inB],
                    [wz_inB]])

# Define roll, pitch, and yaw rotation matrices that describe the bus's orientation in the 
# world frame
Rx = sym.Matrix([[1,            0,             0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi),  sym.cos(phi)]])
Ry = sym.Matrix([[ sym.cos(theta), 0, sym.sin(theta)],
                 [              0, 1,              0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi),  sym.cos(psi), 0],
                 [           0,             0, 1]])
R_ofB_inW = Rz @ Ry @ Rx
R_ofW_inB = R_ofB_inW.T

In [4]:
# Define inputs
taux_inB, tauy_inB, tauz_inB, fz_inB = sym.symbols('tau_x, tau_y, tau_z, f_z')

# Get the net torque vector
tau_inB = sym.Matrix([[taux_inB],
                      [tauy_inB],
                      [tauz_inB]])
tau_inB = sym.simplify(tau_inB)

# Get the gravity force and convert to body coordiantes
grav_inW = sym.Matrix([[0.],
                       [0.],
                       [-mass * g]])
grav_inB = R_ofW_inB @ grav_inW

# Get the net force vector
f_inB = grav_inB + sym.Matrix([[0.],
                               [0.],
                               [fz_inB]])
f_inB = sym.simplify(f_inB)

In [5]:
# Assemble the moment of inertia matrix
I_inB = sym.Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])

In [6]:
v_inW = R_ofB_inW @ v_inB
xyz_dot = sym.simplify(v_inW)
print("px, py, pz rate as a function of orientation and body-fixed velocities:")
sym.N(xyz_dot,3)

px, py, pz rate as a function of orientation and body-fixed velocities:


⎡vx⋅cos(ψ)⋅cos(θ) + vy⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + vz⋅(sin(φ)⋅sin
⎢                                                                             
⎢vx⋅sin(ψ)⋅cos(θ) + vy⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - vz⋅(sin(φ)⋅cos
⎢                                                                             
⎣                            -vx⋅sin(θ) + vy⋅sin(φ)⋅cos(θ) + vz⋅cos(φ)⋅cos(θ) 

(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ))⎤
                           ⎥
(ψ) - sin(ψ)⋅sin(θ)⋅cos(φ))⎥
                           ⎥
                           ⎦

In [7]:
v_inB_dot = (1 / mass) * (f_inB - w_inB.cross(mass * v_inB))
v_inB_dot = sym.simplify(v_inB_dot)
print("Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:")
sym.N(v_inB_dot,3)

Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:


⎡        -ω_y⋅vz + 1.0⋅ω_z⋅vy + 9.81⋅sin(θ)        ⎤
⎢                                                  ⎥
⎢     1.0⋅ωₓ⋅vz - ω_z⋅vx - 9.81⋅sin(φ)⋅cos(θ)      ⎥
⎢                                                  ⎥
⎣1.16⋅f_z - ωₓ⋅vy + 1.0⋅ω_y⋅vx - 9.81⋅cos(φ)⋅cos(θ)⎦

In [8]:
# Define the transformation that takes roll, pitch, and yaw rates to body-fixed angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)

# Invert the transformation to get a matrix that takes body-fixed angular rates to 
# roll, pitch, and yaw rates
M_inv = sym.simplify(M.inv())

# Convert the body-fixed angular rates to roll, pitch, and yaw rates
rpy_dot = sym.simplify(M_inv@w_inB)
print("Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:")
sym.N(rpy_dot,3)

Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:


⎡ωₓ + ω_y⋅sin(φ)⋅tan(θ) + ω_z⋅cos(φ)⋅tan(θ)⎤
⎢                                          ⎥
⎢         ω_y⋅cos(φ) - ω_z⋅sin(φ)          ⎥
⎢                                          ⎥
⎢         ω_y⋅sin(φ) + ω_z⋅cos(φ)          ⎥
⎢         ───────────────────────          ⎥
⎣                  cos(θ)                  ⎦

In [9]:
# Apply Euler's equation to get the time derivative of the angular velocities of the
# quadrotor in the quadrotor frame
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB))
w_inB_dot = sym.simplify(w_inB_dot)
print("Time derivative of the angular velocities in the body-fixed frame:")
sym.N(w_inB_dot, 3)

Time derivative of the angular velocities in the body-fixed frame:


⎡Iyy⋅ω_y⋅ω_z - Izz⋅ω_y⋅ω_z + τₓ⎤
⎢──────────────────────────────⎥
⎢             Ixx              ⎥
⎢                              ⎥
⎢-Ixx⋅ωₓ⋅ω_z + Izz⋅ωₓ⋅ω_z + τ_y⎥
⎢──────────────────────────────⎥
⎢             Iyy              ⎥
⎢                              ⎥
⎢Ixx⋅ωₓ⋅ω_y - Iyy⋅ωₓ⋅ω_y + τ_z ⎥
⎢───────────────────────────── ⎥
⎣             Izz              ⎦

In [10]:
f = sym.Matrix.vstack(xyz_dot,
                      v_inB_dot,
                      rpy_dot,
                      w_inB_dot)

In [11]:
sym.N(f,3)

⎡vx⋅cos(ψ)⋅cos(θ) + vy⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + vz⋅(sin(φ)⋅sin
⎢                                                                             
⎢vx⋅sin(ψ)⋅cos(θ) + vy⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - vz⋅(sin(φ)⋅cos
⎢                                                                             
⎢                            -vx⋅sin(θ) + vy⋅sin(φ)⋅cos(θ) + vz⋅cos(φ)⋅cos(θ) 
⎢                                                                             
⎢                                   -ω_y⋅vz + 1.0⋅ω_z⋅vy + 9.81⋅sin(θ)        
⎢                                                                             
⎢                                1.0⋅ωₓ⋅vz - ω_z⋅vx - 9.81⋅sin(φ)⋅cos(θ)      
⎢                                                                             
⎢                           1.16⋅f_z - ωₓ⋅vy + 1.0⋅ω_y⋅vx - 9.81⋅cos(φ)⋅cos(θ)
⎢                                                                             
⎢                               ωₓ + ω_y⋅sin(φ)⋅tan(